# ディープラーニングによる自然言語処理: 評判分析モデルの実装

このノートブックでは、ディープラーニングを用いた自然言語処理のためのツールである[AllenNLP](https://allennlp.org/)を用いて、**畳み込みニューラルネットワーク（CNN）**を用いた**評判分析**のシステムを作成します。

## 環境のセットアップ

In [ ]:
# AllenNLPをインストール
!pip install allennlp==2.9.2 google-cloud-storage==2.1.0
# fugashiをUniDicの依存ライブラリを含めてインストール
!pip install fugashi[unidic]
# UniDicの辞書ファイルをダウンロード
!python -m unidic download

     |████████████████████████████████| 719 kB 9.9 MB/s 
     |████████████████████████████████| 106 kB 36.6 MB/s 
     |████████████████████████████████| 1.2 MB 29.0 MB/s 
     |████████████████████████████████| 3.8 MB 52.2 MB/s 
     |████████████████████████████████| 77 kB 5.8 MB/s 
     |████████████████████████████████| 125 kB 50.1 MB/s 
     |████████████████████████████████| 248 kB 46.8 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.8 MB 44.2 MB/s 
     |████████████████████████████████| 592 kB 45.9 MB/s 
     |████████████████████████████████| 114 kB 50.2 MB/s 
     |████████████████████████████████| 76 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 46.8 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 8.7 MB 42.5 MB/s 
     |████████████████████

     |████████████████████████████████| 568 kB 12.2 MB/s 
  Created wheel for unidic: filename=unidic-1.1.0-py3-none-any.whl size=7426 sha256=e1a4ee40fc00a60947d6a8a590c1d0ba841d60bae682e682e67c309312f3cc7b
  Stored in directory: /root/.cache/pip/wheels/ce/4d/f1/170bb74b559ca338113c0315c9805e16dfd0a12411ec6b1122
Successfully built unidic
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:45<00:00, 11.7MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.7/dist-packages/unidic/dicdir


## データセットのセットアップ

In [ ]:
# データセットの出力ディレクトリを作成
!mkdir -p data/amazon_reviews
# データセットをダウンロード
!wget -q -O data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv.gz "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_multilingual_JP_v1_00.tsv.gz"
# データセットを解凍し、data/amazon_reviewsに展開
!gunzip data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv.gz

In [ ]:
import csv
import json
import os
import random
import warnings
from bs4 import BeautifulSoup

# csvライブラリのフィールドの最大サイズを変更
csv.field_size_limit(1000000)
# BeautifulSoupの出力する警告を抑制
warnings.filterwarnings("ignore", category=UserWarning, module="bs4")

# データセットをファイルから読み込む
data = []
with open("data/amazon_reviews/amazon_reviews_multilingual_JP_v1_00.tsv") as f:
    reader = csv.reader(f, delimiter="\t")
    # 1行目はヘッダなので無視する
    next(reader)
    for r in reader:
        # レビューのテキストを取得
        review_body = r[13]
        # レビューのテキストからHTMLタグを除去
        review_body = BeautifulSoup(review_body, "html.parser").get_text()
        # 評価の値を取得
        ratings = int(r[7])
        # 評価が2以下の場合に否定的、4以上の場合に肯定的と扱う
        if ratings <= 2:
            data.append(dict(text=review_body, label="negative"))
        elif ratings >= 4:
            data.append(dict(text=review_body, label="positive"))

# データセットから50,000件をランダムに抽出する
random.seed(1)
random.shuffle(data)
data = data[:50000]

# データセットの80%を訓練データ、10%を検証データ、10%をテストデータとして用いる
split_data = {}
eval_size = int(len(data) * 0.1)
split_data["test"] = data[:eval_size]
split_data["validation"] = data[eval_size:eval_size * 2]
split_data["train"] = data[eval_size * 2:]

# 行区切りJSON形式でデータセットを書き込む
for fold in ("train", "validation", "test"):
    out_file = os.path.join("data/amazon_reviews", "amazon_reviews_{}.jsonl".format(fold))
    with open(out_file, mode="w") as f:
        for item in split_data[fold]:
            json.dump(item, f, ensure_ascii=False)
            f.write("\n")

In [ ]:
!ls data/amazon_reviews/*.jsonl

data/amazon_reviews/amazon_reviews_test.jsonl
data/amazon_reviews/amazon_reviews_train.jsonl
data/amazon_reviews/amazon_reviews_validation.jsonl


## モデルの実装

In [ ]:
model_config = """{
    "random_seed": 1,
    "pytorch_seed": 1,
    "train_data_path": "data/amazon_reviews/amazon_reviews_train.jsonl",
    "validation_data_path": "data/amazon_reviews/amazon_reviews_validation.jsonl",
    "dataset_reader": {
        "type": "text_classification_json",
        "tokenizer": {
            "type": "mecab"
        },
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        }
    },
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "validation_data_loader": {
        "batch_size": 32,
        "shuffle": false
    },
    "vocabulary": {},
    "datasets_for_vocab_creation": ["train"],
    "model": {
        "type": "basic_classifier",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": 100
                }
            }
        },
        "seq2vec_encoder": {
           "type": "cnn",
           "embedding_dim": 100,
           "ngram_filter_sizes": [2],
           "num_filters": 64,
           "conv_layer_activation": "relu"
        }
    },
    "trainer": {
        "cuda_device": 0,
        "optimizer": {
            "type": "adam"
        },
        "num_epochs": 10,
        "patience": 3,
        "callbacks": [
            {
                "type": "tensorboard"
            }
        ]
    }
}"""
with open("amazon_reviews.jsonnet", "w") as f:
  f.write(model_config)

## モデルの訓練

In [ ]:
mecab_tokenizer_code = '''from allennlp.data.tokenizers.token_class import Token
from allennlp.data.tokenizers.tokenizer import Tokenizer
from fugashi import Tagger

@Tokenizer.register("mecab")
class MecabTokenizer(Tokenizer):
    def __init__(self):
        # Taggerインスタンスを作成
        self._tagger = Tagger()

    def tokenize(self, text):
        """入力テキストをMeCabを用いて解析する"""
        tokens = []
        # 入力テキストを単語に分割
        for word in self._tagger(text):
            # 単語のテキスト（word.surface）と品詞（word.feature.pos1)からTokenインスタンスを作成
            token = Token(text=word.surface, pos_=word.feature.pos1)
            tokens.append(token)

        return tokens
'''
with open("mecab_tokenizer.py", "w") as f:
  f.write(mecab_tokenizer_code)

In [ ]:
!allennlp train --serialization-dir exp_amazon_reviews --include-package mecab_tokenizer amazon_reviews.jsonnet

2022-04-09 14:17:44,322 - INFO - allennlp.common.params - evaluation = None
2022-04-09 14:17:44,323 - INFO - allennlp.common.params - include_in_archive = None
2022-04-09 14:17:44,323 - INFO - allennlp.common.params - random_seed = 1
2022-04-09 14:17:44,323 - INFO - allennlp.common.params - numpy_seed = 1337
2022-04-09 14:17:44,323 - INFO - allennlp.common.params - pytorch_seed = 1
2022-04-09 14:17:44,391 - INFO - allennlp.common.checks - Pytorch version: 1.10.0+cu111
2022-04-09 14:17:44,392 - INFO - allennlp.common.params - type = default
2022-04-09 14:17:44,392 - INFO - allennlp.common.params - dataset_reader.type = text_classification_json
2022-04-09 14:17:44,393 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2022-04-09 14:17:44,393 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2022-04-09 14:17:44,393 - INFO - allennlp.common.params - dataset_reader.manual_multiprocess_sharding = False
2022-04-09 14:17:44,393 - INFO - all

## 性能の評価

In [ ]:
!allennlp evaluate --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz data/amazon_reviews/amazon_reviews_test.jsonl

2022-04-09 14:20:46,734 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2022-04-09 14:20:46,734 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmpdtu9j9yg
2022-04-09 14:20:47,202 - INFO - allennlp.data.vocabulary - Loading token dictionary from /tmp/tmpdtu9j9yg/vocabulary.
2022-04-09 14:20:47,454 - INFO - allennlp.models.archival - removing temporary unarchived model dir at /tmp/tmpdtu9j9yg
2022-04-09 14:20:47,485 - INFO - allennlp.common.checks - Pytorch version: 1.10.0+cu111
2022-04-09 14:20:47,487 - INFO - allennlp.commands.evaluate - Reading evaluation data from amazon_reviews_test
loading instances: 5000it [00:09, 521.12it/s]
2022-04-09 14:20:57,890 - INFO - allennlp.evaluation.evaluator - Iterating over dataset
accuracy: 0.92, loss: 0.20 ||: : 157it [00:07, 20.81it/s]
2022-04-09 14:21:05,435 - INFO - allennlp.commands.evaluate - Finished evaluating.


## 学習したモデルを使う

In [ ]:
input_json = """{"sentence":"この本は、役に立つし、面白い。"}
{"sentence":"この本は、役に立たないし、面白くない。"}
{"sentence":"この本は、役に立たないけど、面白い。"}
"""
with open("sentiment_analysis_input.json", "w") as f:
  f.write(input_json)

In [ ]:
!allennlp predict --include-package mecab_tokenizer exp_amazon_reviews/model.tar.gz sentiment_analysis_input.json

2022-04-09 14:21:12,927 - INFO - allennlp.models.archival - loading archive file exp_amazon_reviews/model.tar.gz
2022-04-09 14:21:12,928 - INFO - allennlp.models.archival - extracting archive file exp_amazon_reviews/model.tar.gz to temp dir /tmp/tmpf9hpafz6
2022-04-09 14:21:13,413 - INFO - allennlp.common.params - dataset_reader.type = text_classification_json
2022-04-09 14:21:13,413 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2022-04-09 14:21:13,413 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2022-04-09 14:21:13,414 - INFO - allennlp.common.params - dataset_reader.manual_multiprocess_sharding = False
2022-04-09 14:21:13,414 - INFO - allennlp.common.params - dataset_reader.token_indexers.tokens.type = single_id
2022-04-09 14:21:13,414 - INFO - allennlp.common.params - dataset_reader.token_indexers.tokens.namespace = tokens
2022-04-09 14:21:13,414 - INFO - allennlp.common.params - dataset_reader.token_indexers.tokens.low

### ハイパーパラメータ探索

In [ ]:
!pip install optuna

     |████████████████████████████████| 308 kB 12.1 MB/s 
     |████████████████████████████████| 81 kB 7.0 MB/s 
     |████████████████████████████████| 210 kB 51.0 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 150 kB 51.2 MB/s 
     |████████████████████████████████| 113 kB 47.3 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=b0c7635b725e19363ffda65a3a286d4b04610ac1f7956821eec3e338f1316a6a
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
optuna_config = """local embedding_dim = std.parseInt(std.extVar("embedding_dim"));
local num_filters = std.parseInt(std.extVar("num_filters"));
local max_filter_size = std.parseInt(std.extVar("max_filter_size"));
local ngram_filter_sizes = std.range(2, max_filter_size);
{
    "random_seed": 1,
    "pytorch_seed": 1,
    "train_data_path": "data/amazon_reviews/amazon_reviews_train.jsonl",
    "validation_data_path": "data/amazon_reviews/amazon_reviews_validation.jsonl",
    "dataset_reader": {
        "type": "text_classification_json",
        "tokenizer": {
            "type": "mecab"
        },
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        }
    },
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "validation_data_loader": {
        "batch_size": 32,
        "shuffle": false
    },
    "vocabulary": {},
    "datasets_for_vocab_creation": ["train"],
    "model": {
        "type": "basic_classifier",
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "type": "embedding",
                    "embedding_dim": embedding_dim
                }
            }
        },
        "seq2vec_encoder": {
           "type": "cnn",
           "embedding_dim": embedding_dim,
           "ngram_filter_sizes": ngram_filter_sizes,
           "num_filters": num_filters,
           "conv_layer_activation": "relu"
        }
    },
    "trainer": {
        "cuda_device": 0,
        "optimizer": {
            "type": "adam"
        },
        "num_epochs": 10,
        "patience": 3
    }
}
"""
with open("amazon_reviews_optuna.jsonnet", "w") as f:
  f.write(optuna_config)

In [ ]:
import optuna
from optuna.samplers import TPESampler
from optuna.integration.allennlp import AllenNLPExecutor, dump_best_config

def objective(trial):
    """Optunaの目的関数"""
    # 探索するハイパーパラメータと値の範囲を定義
    trial.suggest_int("embedding_dim", 50, 200)
    trial.suggest_int("max_filter_size", 2, 5)
    trial.suggest_int("num_filters", 32, 256)

    # AllenNLPExecutorを作成し訓練と評価を行う
    serialization_dir = "exp_amazon_reviews_optuna/trials/" + str(trial.number)
    executor = AllenNLPExecutor(
        trial=trial,
        metrics="best_validation_accuracy",
        serialization_dir=serialization_dir,
        config_file="amazon_reviews_optuna.jsonnet",
        include_package="mecab_tokenizer"
    )
    return executor.run()

In [ ]:
!mkdir -p exp_amazon_reviews_optuna

In [ ]:
# ハイパーパラメータ探索を開始
study = optuna.create_study(direction="maximize", sampler=TPESampler(seed=1))
study.optimize(objective, n_trials=100)

# 最も性能の良かったハイパーパラメータを用いて設定ファイルを作成
dump_best_config("amazon_reviews_optuna.jsonnet", "amazon_reviews_best.json", study)

[I 2022-04-09 14:21:30,612] A new study created in memory with name: no-name-82d309db-bc89-4ca6-bd30-8f9efab85090


2022-04-09 14:21:30,722 - INFO - allennlp.common.params - random_seed = 1
2022-04-09 14:21:30,726 - INFO - allennlp.common.params - numpy_seed = 1337
2022-04-09 14:21:30,728 - INFO - allennlp.common.params - pytorch_seed = 1
2022-04-09 14:21:30,764 - INFO - allennlp.common.checks - Pytorch version: 1.10.0+cu111
2022-04-09 14:21:30,768 - INFO - allennlp.common.params - type = default
2022-04-09 14:21:30,772 - INFO - allennlp.common.params - dataset_reader.type = text_classification_json
2022-04-09 14:21:30,775 - INFO - allennlp.common.params - dataset_reader.max_instances = None
2022-04-09 14:21:30,778 - INFO - allennlp.common.params - dataset_reader.manual_distributed_sharding = False
2022-04-09 14:21:30,780 - INFO - allennlp.common.params - dataset_reader.manual_multiprocess_sharding = False
2022-04-09 14:21:30,782 - INFO - allennlp.common.params - dataset_reader.token_indexers.tokens.type = single_id
2022-04-09 14:21:30,784 - INFO - allennlp.common.params - dataset_reader.token_index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: ExperimentalWarning: AllenNLPExecutor is experimental (supported from v1.4.0). The interface can change in the future.


2022-04-09 14:21:30,819 - INFO - allennlp.common.params - datasets_for_vocab_creation = ['train']
2022-04-09 14:21:30,831 - INFO - allennlp.common.params - validation_dataset_reader = None
2022-04-09 14:21:30,837 - INFO - allennlp.common.params - validation_data_path = data/amazon_reviews/amazon_reviews_validation.jsonl
2022-04-09 14:21:30,839 - INFO - allennlp.common.params - test_data_path = None
2022-04-09 14:21:30,843 - INFO - allennlp.common.params - evaluate_on_test = False
2022-04-09 14:21:30,846 - INFO - allennlp.common.params - batch_weight_key = 
2022-04-09 14:21:30,848 - INFO - allennlp.common.params - data_loader.type = multiprocess
2022-04-09 14:21:30,851 - INFO - allennlp.common.params - data_loader.batch_size = 32
2022-04-09 14:21:30,855 - INFO - allennlp.common.params - data_loader.drop_last = False
2022-04-09 14:21:30,859 - INFO - allennlp.common.params - data_loader.shuffle = True
2022-04-09 14:21:30,860 - INFO - allennlp.common.params - data_loader.batch_sampler = No

loading instances: 40000it [01:26, 464.43it/s]

...

2022-04-09 21:31:32,386 - INFO - allennlp.training.gradient_descent_trainer - Validating



accuracy: 0.9232, batch_loss: 0.0004, loss: 0.3041 ||: 100%|##########| 157/157 [00:01<00:00, 147.70it/s]

2022-04-09 21:31:33,464 - INFO - allennlp.training.callbacks.console_logger -                        Training |  Validation
2022-04-09 21:31:33,466 - INFO - allennlp.training.callbacks.console_logger - accuracy           |     0.994  |     0.923
2022-04-09 21:31:33,468 - INFO - allennlp.training.callbacks.console_logger - gpu_0_memory_MB    |  1106.606  |       N/A
2022-04-09 21:31:33,470 - INFO - allennlp.training.callbacks.console_logger - loss               |     0.020  |     0.304
2022-04-09 21:31:33,471 - INFO - allennlp.training.callbacks.console_logger - worker_0_memory_MB |  52193.410  |       N/A


2022-04-09 21:31:34,136 - INFO - allennlp.training.gradient_descent_trainer - Epoch duration: 0:00:19.792253
2022-04-09 21:31:34,139 - INFO - allennlp.training.gradient_descent_trainer - Ran out of patience. Stopping training.
2022-04-09 21:31:34,183 - INFO - allennlp.common.util - Metrics: {
  "best_epoch": 0,
  "peak_worker_0_memory_MB": 52193.41015625,
  "peak_gpu_0_memory_MB": 1211.509765625,
  "training_duration": "0:01:21.820769",
  "epoch": 3,
  "training_accuracy": 0.994425,
  "training_loss": 0.02005230436643469,
  "training_worker_0_memory_MB": 52193.41015625,
  "training_gpu_0_memory_MB": 1106.60595703125,
  "validation_accuracy": 0.9232,
  "validation_loss": 0.3041337210285347,
  "best_validation_accuracy": 0.9228,
  "best_validation_loss": 0.19101896791890927
}
2022-04-09 21:31:34,186 - INFO - allennlp.models.archival - archiving weights and vocabulary to exp_amazon_reviews_optuna/trials/99/model.tar.gz


[I 2022-04-09 21:31:37,738] Trial 99 finished with value: 0.9228 and parameters: {'embedding_dim': 146, 'max_filter_size': 3, 'num_filters': 75}. Best is trial 64 with value: 0.9354.


In [ ]:
!cat amazon_reviews_best.json

{
    "data_loader": {
        "batch_size": 32,
        "shuffle": true
    },
    "dataset_reader": {
        "token_indexers": {
            "tokens": {
                "type": "single_id"
            }
        },
        "tokenizer": {
            "type": "mecab"
        },
        "type": "text_classification_json"
    },
    "datasets_for_vocab_creation": [
        "train"
    ],
    "model": {
        "seq2vec_encoder": {
            "conv_layer_activation": "relu",
            "embedding_dim": 170,
            "ngram_filter_sizes": [
                2,
                3,
                4,
                5
            ],
            "num_filters": 50,
            "type": "cnn"
        },
        "text_field_embedder": {
            "token_embedders": {
                "tokens": {
                    "embedding_dim": 170,
                    "type": "embedding"
                }
            }
        },
        "type": "basic_classifier"
    },
    "pytorch_seed": 1,
    "rando